### Introduction
In this code, we calculated the the new denominator by following equation:


```
# Computing the total SNL mines
total_SNL_mines = len(new_df[(new_df['SNL_mines'] == 1)])

# Intersection between SNL mines found and indig = 1
intersect = len(new_df[(new_df['SNL_mines'] == 1) & (new_df['indig'] == 1) ])

# Computing new denominator
new_denominator = total_SNL_mines/intersect
```

In [1]:
import numpy as np
import pandas as pd

### Read Prio Grid data

In [2]:
prio_grid = pd.read_csv("../input/PRIO GRID spine.csv")
prio_grid["gid"] = prio_grid["gid"].astype("int64")
print(prio_grid.shape)
prio_grid.head()

(64818, 3)


gid    lon    lat
0  49182 -69.25 -55.75
1  49183 -68.75 -55.75
2  49184 -68.25 -55.75
3  49185 -67.75 -55.75
4  49186 -67.25 -55.75

### Read indigenous land data

In [3]:
ind_land=pd.read_csv("../input/ind_land.csv",usecols=["GRID IDs","date"])
print(ind_land.shape)
ind_land= ind_land[pd.to_numeric(ind_land['GRID IDs'], errors='coerce').notnull()].reset_index()
ind_land["GRID IDs"] = ind_land["GRID IDs"].astype("int")
ind_land

(159050, 2)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18959/305043807.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  ind_land=pd.read_csv("../input/ind_land.csv",usecols=["GRID IDs","date"])


index  GRID IDs     date
0            0     68612  2021-01
1            1    157122  2021-01
2            2    157122  2021-01
3            3    157122  2021-01
4            4    157122  2021-01
...        ...       ...      ...
156868  159045    160720  2021-12
156869  159046    160720  2021-12
156870  159047    157122  2021-12
156871  159048    175815  2021-12
156872  159049    157124  2021-12

[156873 rows x 3 columns]

#### Find the the count of indigenous land on each PRIO GRID cell

In [4]:
ind_land_count = pd.DataFrame(ind_land.groupby("GRID IDs")["index"].count()).reset_index().rename(columns={"index":"count"})
ind_land_count.head()


GRID IDs  count
0     68612     22
1     68614     13
2     69334     73
3     69336     15
4     71496     47

In [10]:
ind_grid = prio_grid.merge(ind_land_count, left_on="gid", right_on="GRID IDs", how="left").fillna(0).drop(columns=["GRID IDs","lon","lat"])
ind_grid.head()

gid  count
0  49182    0.0
1  49183    0.0
2  49184    0.0
3  49185    0.0
4  49186    0.0

### Read SNL data

In [11]:
df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")
print(df_SNL.shape)
df_SNL.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
df_SNL = df_SNL[pd.to_numeric(df_SNL['PRIO GRID ID'], errors='coerce').notnull()]
df_SNL["PRIO GRID ID"] = df_SNL["PRIO GRID ID"].astype("int")
df_SNL = df_SNL[(df_SNL['LATITUDE']<=90) & (df_SNL['LATITUDE']>= -90) &
                (df_SNL['LONGITUDE']<=180) & (df_SNL['LONGITUDE']>= -180)].reset_index().drop(columns="index",axis=1)
print(df_SNL.shape)
df_SNL.head()

(39612, 46)
(39370, 46)


/var/folders/0p/mkzn2l513fl51bl6rs96d9z40000gn/T/ipykernel_18959/2275177157.py:1: DtypeWarning: Columns (2,22,23,24,25,26,27,28,30,31,32,33,36,37,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df_SNL= pd.read_csv("../input/SNL-PRIO GRID IDs.csv")


Unnamed: 0           Property name Property ID Also Known As  \
0          0           11 RAU-Novaya       68289           NaN   
1          1           12 Mile Creek       66178           NaN   
2          2               14 Karats       65525           NaN   
3          3  14 Mile Well Extension       63665           NaN   
4          4     15 Mile Silver City       65760           NaN   

                                     OWNER(S) Equity ownership %  \
0  Sodeistvie Private Enterprise (Owner) 100%                100   
1               U&D Coal Limited (Owner) 100%                100   
2          Dios Exploration Inc. (Owner) 100%                100   
3               Private Interest (Owner) 100%                100   
4       North Lily Mining Company (Owner) 50%                 50   

  DEVELOPMENT STAGE ACTIVITY STATUS   COMMODITY(S) PRIMARY COMMODITY  ...  \
0        Grassroots        Inactive           Coal              Coal  ...   
1        Grassroots        Inactive           Coal              Coal  ...   
2        Grassroots        Inactive  Gold,Diamonds              Gold  ...   
3        Grassroots        Inactive           Gold              Gold  ...   
4        Grassroots        Inactive    Gold,Silver              Gold  ...   

  Unnamed: 36 Unnamed: 37 Unnamed: 38 Unnamed: 39 Unnamed: 40  Unnamed: 41  \
0         NaN         NaN         NaN         NaN         NaN          NaN   
1         NaN         NaN         NaN         NaN         NaN          NaN   
2         NaN         NaN         NaN         NaN         NaN          NaN   
3         NaN         NaN         NaN         NaN         NaN          NaN   
4         NaN         NaN         NaN         NaN         NaN          NaN   

   Unnamed: 42 Unnamed: 43 Unnamed: 44 PRIO GRID ID  
0          NaN         NaN         NaN       199878  
1          NaN         NaN         NaN        96419  
2          NaN         NaN         NaN       205418  
3          NaN         NaN         NaN        88445  
4          NaN         NaN         NaN       221841  

[5 rows x 46 columns]

Find the the count of indigenous land on each PRIO GRID cell

In [13]:
snl_count = pd.DataFrame(df_SNL.groupby("PRIO GRID ID")["Property ID"].count()).reset_index().rename(columns={"Property ID":"count"})
snl_count.head()

PRIO GRID ID  count
0         50619      1
1         50624      1
2         52063      1
3         52776      1
4         52778      1

In [16]:
snl_grid = prio_grid.merge(snl_count, left_on="gid", right_on="PRIO GRID ID", how="left").fillna(0).drop(columns=["PRIO GRID ID","lon","lat"])
snl_grid.head()

gid  count
0  49182    0.0
1  49183    0.0
2  49184    0.0
3  49185    0.0
4  49186    0.0

### Merge the SNL dataset with Indigenous Land dataset

In [17]:
snl_ind = snl_grid.merge(ind_grid, left_on="gid", right_on="gid", suffixes=["_snl", "_ind"])
snl_ind

gid  count_snl  count_ind
0       49182        0.0        0.0
1       49183        0.0        0.0
2       49184        0.0        0.0
3       49185        0.0        0.0
4       49186        0.0        0.0
...       ...        ...        ...
64813  249340        0.0        0.0
64814  249341        0.0        0.0
64815  249342        0.0        0.0
64816  249343        0.0        0.0
64817  249344        0.0        0.0

[64818 rows x 3 columns]

### Calculate the the denominator

The new denominator for all SNL data is 22.88

In [18]:
new_value = snl_ind[(snl_ind["count_snl"]>=1)].shape[0]/ snl_ind[(snl_ind["count_snl"]>=1)& snl_ind["count_ind"]>=1].shape[0]
new_value

22.886654478976233